# **Accessing and working with ICESat-2 data in the cloud**


<div align="left"><img src='https://raw.githubusercontent.com/nsidc/NSIDC-Data-Tutorials/is2-cryo2/notebooks/ICESat-2_Cloud_Access/img/nsidc_logo.png'/></div>
   

## **1. Tutorial Overview**

This notebook demonstrates searching for cloud-hosted ICESat-2 data and directly accessing Land Ice Height (ATL06) granules from an Amazon Compute Cloud (EC2) instance using the `earthaccess` package. NASA data "in the cloud" are stored in Amazon Web Services (AWS) Simple Storage Service (S3) Buckets. **Direct Access** is an efficient way to work with data stored in an S3 Bucket when you are working in the cloud. Cloud-hosted granules can be opened and loaded into memory without the need to download them first. This allows you take advantage of the scalability and power of cloud computing. 

The Amazon Global cloud is divided into geographical regions.  To have direct access to data stored in a region, our compute instance - a virtual computer that we create to perform processing operations in place of using our own desktop or laptop - must be in the same region as the data.  This is a fundamental concept of _analysis in place_. **NASA cloud-hosted data is in Amazon Region us-west2.  So your compute instance must also be in us-west2.** If we wanted to use data stored in another region, to use direct access for that data, we would start a compute instance in that region.

As an example data  collection, we use ICESat-2 Land Ice Height (ATL06) over the Juneau Icefield, AK, for March 2003. ICESat-2 data granules, including ATL06, are stored in HDF5 format. We demonstrate how to open an HDF5 granule and access data variables using `xarray`. Land Ice Heights are then plotted using `hvplot`. 

`earthaccess` is a community-developed Python package developed by Luis Lopez (NSIDC developer) to allow easy search of the NASA Common Metadata Repository (CMR) and download of NASA data collections.  It can be used for programmatic search and access for both _on-premises-hosted_ and _cloud-hosted_ data. It manages authenticating using Earthdata Login credentials which are then used to obtain the S3 tokens that are needed for S3 direct access. https://github.com/nsidc/earthaccess


### **Credits**

The notebook was created by Andy Barrett, NSIDC, updated by Jennifer Roebuck and Amy Steiker, NSIDC, and is based on notebooks developed by Luis Lopez and Mikala Beig, NSIDC.

For questions regarding the notebook, or to report problems, please create a new issue in the [NSIDC-Data-Tutorials repo](https://github.com/nsidc/NSIDC-Data-Tutorials/issues).

### **Learning Objectives**

By the end of this demonstration you will be able to:  
1. use `earthaccess` to search for ICESat-2 data using spatial and temporal filters and explore search results;  
2. open data granules using direct access to the ICESat-2 S3 bucket;  
3. load HDF5 data into an `xarray.Datatree` object;
4. visualize the land ice heights using `hvplot`.  

### **Prerequisites**

1. An EC2 instance in the us-west-2 region. **NASA cloud-hosted data is in Amazon Region us-west2. So you also need an EC2 instance in the us-west-2 region.** An EC2 instance is a virtual computer that you create to perform processing operations in place of using your own desktop or laptop.  Details on how to set up an instance can be found [here](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide#anchor-1).
2. An Earthdata Login is required for data access. If you don't have one, you can register for one [here](https://urs.earthdata.nasa.gov/).
3. A .netrc file, that contains your Earthdata Login credentials, in your home directory. The current recommended practice for authentication is to create a .netrc file in your home directory following [these instructions](https://nsidc.org/support/how/how-do-i-programmatically-request-data-services) (Step 1) and to use the .netrc file for authentication when required for data access during the tutorial.
4. The *nsidc-tutorials* environment is setup and activated. This [README](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/README.md) has setup instructions.

### **Example of end product** 
At the end of this tutorial, the following figure will be generated:
       
![ATL06 land ice heights](./img/atl06_example_end_product.png)
### **Time requirement**

Allow approximately 20 minutes to complete this tutorial.

## **2. Tutorial steps**

## Import Packages

The first step in any `python` script or notebook is to import packages.  This tutorial requires the following packages:
- `earthaccess`, which enables Earthdata Login authentication and retrieves AWS credentials; enables collection and granule searches; and S3 access;
- `xarray`, used to load data;
- `hvplot`, used to visualize land ice height data.

We are going to import the whole `earthaccess` package.

We will also import the whole `xarray` package but use a standard short name `xr`, using the `import <package> as <short_name>` syntax.  We could use anything for a short name but `xr` is an accepted standard that most `xarray` users are familiar with.

We only need the `xarray` module from `hvplot` so we import that using the `import <package>.<module>` syntax.

In [1]:
# For searching NASA data
import earthaccess

# For reading data, analysis and plotting
import xarray as xr
import hvplot.xarray
import pprint

## Authenticate

The first step is to get the correct authentication that will allow us to get _cloud-hosted_ ICESat-2 data.  This is all done through Earthdata Login.  The `login` method also gets the correct AWS credentials.

Login requires your Earthdata Login username and password. The `login` method will automatically search for these credentials as environment variables or in a `.netrc` file, and if those aren't available it will prompt us to enter our username and password. We use a `.netrc` strategy. A `.netrc` file is a text file located in our home directory that contains login information for remote machines.  If we don't have a `.netrc` file, `login` can create one for us.

```
earthaccess.login(strategy='interactive', persist=True)
```

In [2]:
auth = earthaccess.login()

## Search for ICESat-2 Collections

`earthaccess` leverages the Common Metadata Repository (CMR) API to search for collections and granules.  [Earthdata Search](https://search.earthdata.nasa.gov/search) also uses the CMR API.

We can use the `search_datasets` method to search for ICESat-2 collections by setting `keyword='ICESat-2'`.

This will display the number of data collections (data sets) that meet this search criteria.

In [3]:
Query = earthaccess.search_datasets(keyword = 'ICESat-2')

Several dozen collections with the keyword ICESat-2 are returned in the Query object.

The `search_datasets` method returns a python list of `DataCollection` objects. We can view the metadata for each collection in long form by passing a `DataCollection` object to print or as a summary using the `summary` method. We can also use the `pprint` function to Pretty Print each object.

We will do this for the first 10 results (objects).

In [4]:
for collection in Query[:10]:
    pprint.pprint(collection.summary(), sort_dicts=True, indent=4)
    print('')
    

{   'concept-id': 'C2559919423-NSIDC_ECS',
    'file-type': "[{'FormatType': 'Native', 'Format': 'HDF5', "
                 "'FormatDescription': 'HTTPS'}]",
    'get-data': [   'https://n5eil01u.ecs.nsidc.org/ATLAS/ATL03.006/',
                    'https://search.earthdata.nasa.gov/search?q=ATL03+V006',
                    'https://openaltimetry.earthdatacloud.nasa.gov/data',
                    'https://nsidc.org/data/data-access-tool/ATL03/versions/6/'],
    'short-name': 'ATL03',
    'version': '006'}

{   'cloud-info': {   'Region': 'us-west-2',
                      'S3BucketAndObjectPrefixNames': [   'nsidc-cumulus-prod-protected/ATLAS/ATL03/006',
                                                          'nsidc-cumulus-prod-public/ATLAS/ATL03/006'],
                      'S3CredentialsAPIDocumentationURL': 'https://data.nsidc.earthdatacloud.nasa.gov/s3credentialsREADME',
                      'S3CredentialsAPIEndpoint': 'https://data.nsidc.earthdatacloud.nasa.gov/s3credentials'}

For each collection, `summary` returns a subset of fields from the collection metadata and the Unified Metadata Model (UMM):
- `concept-id` is a unique id for the collection. It consists of an alphanumeric code and the provider-id specific to the DAAC (Distributed Active Archive Center). You can use the `concept_id` to search for data granules.
- `short_name` is a quick way of referring to a collection (instead of using the full title). It can be found on the collection landing page underneath the collection title after 'DATA SET ID'. See the table below for a list of the shortnames for ICESat-2 collections.
- `version` is the version of each collection.
- `file-type` gives information about the file format of the collection granules.
- `get-data` is a collection of URLs that can be used to access the data, collection landing pages and data tools. 
- `cloud-info` this is for cloud-hosted data and provides additional information about the location of the S3 bucket that holds the data and where to get temporary AWS S3 credentials to access the S3 buckets. `earthaccess` handles these credentials and the links to the S3 buckets, so in general you won't need to worry about this information. 

For the ICESat-2 search results, within the concept-id, there is a provider-id; `NSIDC_ECS` and `NSIDC_CPRD`. `NSIDC_ECS` which is for the _on-prem_ collections and `NSIDC_CPRD` is for the _cloud-hosted_ collections. 

For ICESat-2, `ShortNames` are generally how different products are referred to.

| ShortName | Product Description |
|:-----------:|:---------------------|
| ATL03 | ATLAS/ICESat-2 L2A Global Geolocated Photon Data |
| ATL06 | ATLAS/ICESat-2 L3A Land Ice Height |
| ATL07 | ATLAS/ICESat-2 L3A Sea Ice Height |
| ATL08 | ATLAS/ICESat-2 L3A Land and Vegetation Height |
| ATL09 | ATLAS/ICESat-2 L3A Calibrated Backscatter Profiles and Atmospheric Layer Characteristics |
| ATL10 | ATLAS/ICESat-2 L3A Sea Ice Freeboard |
| ATL11 | ATLAS/ICESat-2 L3B Slope-Corrected Land Ice Height Time Series |
| ATL12 | ATLAS/ICESat-2 L3A Ocean Surface Height |
| ATL13 | ATLAS/ICESat-2 L3A Along Track Inland Surface Water Data |

### Search for cloud-hosted data
For most collections, to search for only data in the cloud, the `cloud_hosted` method can be used. 

In [5]:
Query = earthaccess.search_datasets(
    keyword = 'ICESat-2',
    cloud_hosted = True
)

## Search a data set using spatial and temporal filters 

We can use the `search_data` method to search for granules within a data set by location and time using spatial and temporal filters. In this example, we will search for data granules from the ATL06 verison 006 cloud-hosted data set over the Juneau Icefield, AK, for March and April 2020.

The temporal range is identified with standard date strings, and latitude-longitude corners of a bounding box is specified.  Polygons and points, as well as shapefiles can also be specified.

This will display the number of granules that match our search. 

In [6]:
results = earthaccess.search_data(
    short_name = 'ATL06',
    version = '006',
    cloud_hosted = True,
    bounding_box = (-134.7,58.9,-133.9,59.2),
    temporal = ('2020-03-01','2020-04-30'),
)

To display the rendered metadata, including the download link, granule size and two images, we will use `display`. In the example below, all 4 results are shown. 

The download link is `https` and can be used download the granule to your local machine.  This is similar to downloading _DAAC-hosted_ data but in this case the data are coming from the Earthdata Cloud.  For NASA data in the Earthdata Cloud, there is no charge to the user for egress from AWS Cloud servers.  This is not the case for other data in the cloud.

Note the `[None, None, None, None]` that is displayed at the end can be ignored, it has no meaning in relation to the metadata.

In [7]:
[display(r) for r in results]

Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Land Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -134.3399, 'Latitude': 59.03152}, {'Longitude': -134.44371, 'Latitude': 59.03709}, {'Longitude': -134.75456, 'Latitude': 57.4161}, {'Longitude': -134.6551, 'Latitude': 57.41076}, {'Longitude': -134.3399, 'Latitude': 59.03152}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-03-10T12:15:10.646Z', 'EndingDateTime': '2020-03-10T12:15:58.724Z'}}
Size(MB): 3.034775733947754
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2020/03/10/ATL06_20200310121504_11420606_006_01.h5']

Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Land Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -134.41459, 'Latitude': 59.54887}, {'Longitude': -134.63538, 'Latitude': 59.53739}, {'Longitude': -134.59212, 'Latitude': 59.32438}, {'Longitude': -134.2125, 'Latitude': 57.38652}, {'Longitude': -133.7114, 'Latitude': 54.60434}, {'Longitude': -133.55731, 'Latitude': 53.68969}, {'Longitude': -133.36771, 'Latitude': 53.70055}, {'Longitude': -133.5178, 'Latitude': 54.61543}, {'Longitude': -134.00456, 'Latitude': 57.3981}, {'Longitude': -134.37249, 'Latitude': 59.33607}, {'Longitude': -134.41459, 'Latitude': 59.54887}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-03-12T23:40:36.093Z', 'EndingDateTime': '2020-03-12T23:42:07.293Z'}}
Size(MB): 29.99279499053955
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2020/03/12/ATL06_20200312233336_11800602_006_01.h5']

Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Land Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -134.28339, 'Latitude': 59.54228}, {'Longitude': -134.50433, 'Latitude': 59.53072}, {'Longitude': -134.46626, 'Latitude': 59.34436}, {'Longitude': -133.988, 'Latitude': 56.87365}, {'Longitude': -133.46142, 'Latitude': 53.88745}, {'Longitude': -133.34478, 'Latitude': 53.18986}, {'Longitude': -133.15724, 'Latitude': 53.20085}, {'Longitude': -133.27098, 'Latitude': 53.8986}, {'Longitude': -133.78272, 'Latitude': 56.88519}, {'Longitude': -134.24631, 'Latitude': 59.35609}, {'Longitude': -134.28339, 'Latitude': 59.54228}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-04-10T22:16:27.813Z', 'EndingDateTime': '2020-04-10T22:18:06.791Z'}}
Size(MB): 32.94972229003906
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2020/04/10/ATL06_20200410220936_02350702_006_02.h5']

Collection: {'EntryTitle': 'ATLAS/ICESat-2 L3A Land Ice Height V006'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -134.62761, 'Latitude': 59.53143}, {'Longitude': -134.84844, 'Latitude': 59.54238}, {'Longitude': -134.8761, 'Latitude': 59.3961}, {'Longitude': -135.19822, 'Latitude': 57.71464}, {'Longitude': -135.43039, 'Latitude': 56.42147}, {'Longitude': -135.46534, 'Latitude': 56.22639}, {'Longitude': -135.26355, 'Latitude': 56.21509}, {'Longitude': -135.22779, 'Latitude': 56.41013}, {'Longitude': -134.98845, 'Latitude': 57.70329}, {'Longitude': -134.65611, 'Latitude': 59.38441}, {'Longitude': -134.62761, 'Latitude': 59.53143}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2020-04-12T10:42:46.111Z', 'EndingDateTime': '2020-04-12T10:43:37.436Z'}}
Size(MB): 20.340317726135254
Data: ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2020/04/12/ATL06_20200412104246_02580706_006_02.h5']

[None, None, None, None]

## Use Direct-Access to open, load and display data stored on S3

Direct-access to data from an S3 bucket is a two step process.  First, the files are opened using the `open` method.  The `auth` object created at the start of the notebook is used to provide Earthdata Login authentication and AWS credentials. 

In [8]:
files = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/4 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/4 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/4 [00:00<?, ?it/s]

The next step is to load the data. `xarray.DataTree` objects allow us to work with hierarchical data structures and file formats such as HDF5, Zarr and NetCDF4 with groups. 

We use `xr.open_datatree` to open the ATL06 data. We add the `phony_dims="sort"` option because data variables in several groups including `ancillary_data` do not have any assigned dimension scales. `xarray` names dimensions `phony_dim0`, `phony_dim1`, etc.

In [9]:
dt = xr.open_datatree(files[1], phony_dims='sort')
dt

<xarray.DataTree>
Group: /
│   Attributes: (12/48)
│       short_name:                         ATL06
│       level:                              L3A
│       title:                              ATLAS/ICESat-2 L3A Land Ice Height
│       description:                        This data set (ATL06) provides geoloc...
│       Conventions:                        CF-1.6
│       contributor_name:                   Thomas A Neumann (thomas.neumann@nasa...
│       ...                                 ...
│       processing_level:                   3A
│       references:                         http://nsidc.org/data/icesat2/data.html
│       project:                            ICESat-2 > Ice, Cloud, and land Eleva...
│       instrument:                         ATLAS > Advanced Topographic Laser Al...
│       platform:                           ICESat-2 > Ice, Cloud, and land Eleva...
│       source:                             Spacecraft
├── Group: /ancillary_data
│   │   Dimensions:              (phony_dim_0: 1)
│   │   Dimensions without coordinates: phony_dim_0
│   │   Data variables: (12/25)
│   │       atlas_sdp_gps_epoch  (phony_dim_0) datetime64[ns] 8B ...
│   │       control              (phony_dim_0) |S100000 100kB ...
│   │       data_end_utc         (phony_dim_0) |S27 27B ...
│   │       data_start_utc       (phony_dim_0) |S27 27B ...
│   │       end_cycle            (phony_dim_0) int32 4B ...
│   │       end_delta_time       (phony_dim_0) datetime64[ns] 8B ...
│   │       ...                   ...
│   │       start_gpssow         (phony_dim_0) timedelta64[ns] 8B ...
│   │       start_gpsweek        (phony_dim_0) int32 4B ...
│   │       start_orbit          (phony_dim_0) int32 4B ...
│   │       start_region         (phony_dim_0) int32 4B ...
│   │       start_rgt            (phony_dim_0) int32 4B ...
│   │       version              (phony_dim_0) |S80 80B ...
│   │   Attributes:
│   │       Description:  Contains information ancillary to the data product. This ma...
│   │       data_rate:    Data within this group pertain to the granule in its entirety.
│   └── Group: /ancillary_data/land_ice
│           Dimensions:          (phony_dim_1: 1, phony_dim_2: 6)
│           Dimensions without coordinates: phony_dim_1, phony_dim_2
│           Data variables: (12/24)
│               dt_hist          (phony_dim_1) timedelta64[ns] 8B ...
│               fit_maxiter      (phony_dim_1) int32 4B ...
│               fpb_maxiter      (phony_dim_1) int32 4B ...
│               max_res_ids      (phony_dim_1) int32 4B ...
│               min_dist         (phony_dim_1) float32 4B ...
│               min_gain_th      (phony_dim_1) float32 4B ...
│               ...               ...
│               qs_lim_sss       (phony_dim_1) int32 4B ...
│               rbin_width       (phony_dim_1) float32 4B ...
│               sigma_beam       (phony_dim_1) float32 4B ...
│               sigma_tx         (phony_dim_2) timedelta64[ns] 48B ...
│               t_dead           (phony_dim_2) timedelta64[ns] 48B ...
│               txp_maxiter      (phony_dim_1) int32 4B ...
│           Attributes:
│               Description:  Contains land-ice-specific  information ancillary to the da...
├── Group: /orbit_info
│       Dimensions:                (phony_dim_5: 11, crossing_time: 1, sc_orient_time: 1)
│       Coordinates:
│         * crossing_time          (crossing_time) datetime64[ns] 8B 2020-03-12T23:26...
│         * sc_orient_time         (sc_orient_time) datetime64[ns] 8B 2020-03-11T23:3...
│       Dimensions without coordinates: phony_dim_5
│       Data variables:
│           bounding_polygon_lat1  (phony_dim_5) float64 88B ...
│           bounding_polygon_lon1  (phony_dim_5) float64 88B ...
│           cycle_number           (crossing_time) int8 1B ...
│           lan                    (crossing_time) float64 8B ...
│           orbit_number           (crossing_time) uint16 2B ...
│           rgt                    (crossing_time) int16 2B ...
│        

We can see from the representation of the `xarray.DataTree` object `dt` that there are ten groups in the top, or "root", level. Clicking on Groups reveals various Metadata and Ancillary data groups as well as groups representing each of the left and right beam pairs from the ICESat-2 ATLAS instrument*. We can also see that there are no dimensions, coordinates, or data variables in the root group. Reading the data into `numpy` arrays or a `pandas.Dataframe` could be an alternative method to using `xarray.Datatree`. However, each granule (file) would have to be read first using a package that reads HDF5 files such as `h5py`. `xarray` does this all under-the-hood in a single line.

*ICESat-2 measures photon returns from 3 beam pairs numbered 1, 2 and 3 that each consist of a left and a right beam. In this case, we are interested in plotting the left ground track (gt) of beam pair 1. 

`hvplot` is an interactive plotting tool that is useful for exploring data.

In [10]:
dt['/gt1l/land_ice_segments/h_li'].hvplot(kind='scatter', s=2)

:Scatter   [longitude]   (Land Ice height)

## **3. Learning outcomes recap**

We have learned how to:
1. use `earthaccess` to search for ICESat-2 data using spatial and temporal filters and explore search results;
2. open data granules using direct access to the ICESat-2 S3 bucket;
3. load a HDF5 group into an xarray.Datatree;
4. visualize the land ice heights using hvplot.

## **4. Additional resources**

For general information about NSIDC DAAC data in the Earthdata Cloud: 

[FAQs About NSIDC DAAC's Earthdata Cloud Migration](https://nsidc.org/data/user-resources/help-center/faqs-about-nsidc-daacs-earthdata-cloud-migration)

[NASA Earthdata Cloud Data Access Guide](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide)

Additional tutorials and How Tos:

[NASA Earthdata Cloud Cookbook](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/)